In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from datetime import datetime,timedelta, date
import os, sys, timeit

def FindElement():
    global CurrentDate
    global todaystring
    global RealRange
    global path
    
    try:
        DaySelector(CurrentDate)
        RealRange+=1

    except:
        print("Error locating the data of "+ todaystring + ". The Total time was: ",TotalTime)
        os.system(path)
        print(CurrentDate)
        
def MonthSelector(date):
    
    month = str(date.strftime("%B") + ' ' + str(date.year))

    month_list = soup.select('option')
    

    for i in range(len(month_list)):
        month_list[i] = month_list[i].get_text()


    try:
        index = str(month_list.index(month) + 1 )

        xpath = "//option[.='"+ month + "']"
        monthfind = wd.find_element_by_xpath(xpath)
        monthfind.click()
        
    except:
        print("There is no more data at that month")
        exit()


def DaySelector(date):
    day = str(date.day)  + ' ' + date.strftime("%B")+ ' ' + str(date.year)
    '''
    day_list = list()
    month_list = soup.select('option')
    
    for i in range(len(month_list)):
        month_list[i] = month_list[i].get_text()
    
    for element in month_list:
        if element.count(' ') == 2:
            day_list.append(element)
            
    index = str(day_list.index(day) - 1)
    '''

    xpath = "//option[.='" + day + "']"
    dayfind = wd.find_element_by_xpath(xpath)     
    dayfind.click()

In [ ]:
start = timeit.default_timer()

# Things that you should care about
StartDate = datetime(YEAR,MONTH,DAY)
Country = 'mongolia'  ## Insert the name of the country
City = 'ulaanbaatar'  ## Insert the name of the city
path = 'PATH/OF/THE/FIILE.py'
ChromePath = 'PATH/OF/THE/CHROMEDRVIER/chromedriver'

DateRange = 365 
WorkStart = 8
WorkEnd   = 18

CurrentDate = StartDate
EndDate = CurrentDate + timedelta(days = DateRange -1)
RealRange = 0
TotalTime = 0

print("Collecting data from "+ str(CurrentDate)[:10] + " to "+  str(EndDate)[:10] + ' of ' + City + ', ' + Country + '.')

## Access the main page
url = "https://www.timeanddate.com/weather/"+ Country + "/" + City + "/historic"
wd = webdriver.Chrome(ChromePath)
wd.get(url)
req = wd.page_source
soup = BeautifulSoup(req, 'lxml')

## Select the starting month
MonthSelector(CurrentDate)


while 1:
    
    req = wd.page_source # selenium을 쓸 때 이걸 써주면 beautifulsoup 사용가능
    soup = BeautifulSoup(req,"lxml")

    # Finding the day of interest
    FindElement()

    # Specify the day the program is looking at
    todaystring = soup.select('#wt-his-title')[0].get_text()
    print(todaystring)
    todaylist = todaystring.split(" ")
    todaylist = todaylist[-3:]
    
    month = datetime.strptime(todaylist[1], "%B").month
    
    today = datetime(int(todaylist[2]) , month , int(todaylist[0]) )

    
    # Download the hourly weather data
    table = soup.select('#wt-his') 
    table = str(table)
    table_df_list = pd.read_html(table)
    table_df = table_df_list[0]
    table_df = table_df.iloc[2:-1]       ## Filter the data, only hour and temperature data are left.
    table_df = pd.concat((table_df.iloc[:,0],table_df.iloc[:,2]),axis=1)
    table_df = table_df.dropna()         ## Filter out the NaN data

    # Extract the integers of the temperature data into the reference of Celsius
    
    celsius = '\xa0°C'
    farenheit = '\xa0°F'

    for i in range(table_df.shape[0]):
        
        # When the data is based on Celsius:
        if table_df.iloc[i,1][-3:] == celsius:
            temp_str = table_df.iloc[i,1].replace( celsius , '')
            table_df.iloc[i,1] = int(temp_str)
            
        # When the data is based on Farenheit:
        elif table_df.iloc[i,1][-3:] == farenheit:
            temp_str = table_df.iloc[i,1].replace( farenheit , '')
            table_df.iloc[i,1] = (int(temp_str)-32) * 5/9

    # Extract the integers of the time data in order for the comparison 
    
    for j in range(table_df.shape[0]):
        time_str = table_df.iloc[j,0].replace(':','')     ## Shift from 6:00 into 600
        time_str = time_str[:7]                           ## Sometimes the hour string is brought with the day and month string 
        
      # When the data is shown in 6:00 AM
        if time_str[-2:] == 'am':
            time_int = int(time_str.replace(' am',''))
            table_df.iloc[j,0] = time_int
            
            # 12:30 AM should be changed into 00:30
            if time_int >= 1200:
                table_df.iloc[j,0] = int(time_int) - 1200
                
      # When the data is shown in 6:00 PM
        elif time_str[-2:] == 'pm':
            time_int = int(time_str.replace(' pm',''))
            if time_int < 1200:
                table_df.iloc[j,0] = time_int + 1200
            else:
                table_df.iloc[j,0] = time_int
                
      # When the data is shown in 24:00 time reference
        else:
            table_df.iloc[j,0] = int(time_str)

        # 중복된 시간대를 없애기 위해 내림시킨다.
      
        table_df.iloc[j,0] = int(float(table_df.iloc[j,0]))//100


    # 시간이 중복된 행은 앞 행이 남도록 제거한다.
    table_df.columns = ['Time','Celsius']
    table_df = table_df.drop_duplicates("Time", keep = 'first')
    
    try:
        table_np = np.array(table_df)
        Starttime = np.where(table_np >= WorkStart)[0][0]
        Endtime = np.where(table_np >= WorkEnd)[0][0]
          
        table_df = table_df.iloc[Starttime:Endtime]
      # table_df['Celsius']<0 자체가 하나의 변수이다. "Celsious" 열의 값들이 0보다 작으면 True, 그렇지 않으면 False. 하나의 열만 뽑은 것이므로 당연히 1xn꼴이다.
        table_df = table_df[table_df['Celsius']<0]
      # True = 1이므로 True 값을을 모두 더하면 True인 공간들의 개수를 구하는 것과 같다. 그래서 boolean이 이렇게 유용한갑다.
        time = (table_df['Celsius']<0).sum()

        TotalTime += time
        DayAfter = CurrentDate + timedelta(days = 1)
    except:
        DayAfter = CurrentDate + timedelta(days = 1)
    

    if DayAfter.month > CurrentDate.month or DayAfter.year > CurrentDate.year:
        CurrentDate = DayAfter
        MonthSelector(CurrentDate)
        
    else:
        CurrentDate = DayAfter
        
    if (today - EndDate).days >= 0  : 
        break
    

ActiveRate = TotalTime  * 100 / (RealRange * (WorkEnd - WorkStart))
ActiveRate = round(ActiveRate,2)
Alternation = round(TotalTime * DateRange / RealRange * 720,0)

terminate_time = timeit.default_timer()
time_consumed = round(terminate_time - start,2)

print("\n Time consumed: ", time_consumed, " seconds")
print("\n Switch active rate = ",ActiveRate,"%")
print("\n Switch will alternate ", int(Alternation), " times")
